# Plot patchSeq spatial coordinates

In [2]:
#plot type 1 vs type 2

import os
import pandas as pd
import numpy as np
import napari
import trimesh
import matplotlib.pyplot as plt

# Define file paths
BASE_DIR = os.path.abspath("/Users/yoh/Library/CloudStorage/OneDrive-AllenInstitute/data")
# path for patchseq coordinate file
COORDINATES_FILE = os.path.abspath("/Users/yoh/Downloads/250121_Brians_coordinates2.csv")
# path for type 1 vs type 2
PATCHSEQ_FILE = os.path.abspath("/Users/yoh/Downloads/patchSeqCells.xlsx")

#specify path for 
MESH_FILES = {
    "LC Mesh": os.path.join(BASE_DIR, "LC_mask_2/maskv1/LC_ccf_v1_250102.obj"),
    "subCD Mesh": os.path.join(BASE_DIR, "LC_mask_2/maskv1/subCD_ccf_v1_250102.obj"),
    "subCV Mesh": os.path.join(BASE_DIR, "LC_mask_2/maskv1/subCV_ccf_v1_250102.obj"),
    "MouseBrainAllen3 Mesh": os.path.abspath("/Users/yoh/Desktop/Allen_CCF/MouseBrainAllen3.obj"),
}

# Load the coordinate data
print(f"Loading coordinates from {COORDINATES_FILE}...")
coords_df = pd.read_csv(COORDINATES_FILE)
coords_df = coords_df.dropna(subset=['x', 'y', 'z'])  # Remove rows with missing values
coords_df.columns = coords_df.columns.str.strip()  # Remove trailing spaces
coords_df[['x', 'y', 'z']] /= 25.0  # Convert to voxel space

# Load the patchSeqCells data
print(f"Loading patchSeqCells data from {PATCHSEQ_FILE}...")
patchseq_df = pd.read_excel(PATCHSEQ_FILE)
patchseq_df.columns = patchseq_df.columns.str.strip()
patchseq_df = patchseq_df.rename(columns={'cell_id': 'cell_specimen_id'})

# Merge coordinates with patchSeqCells based on 'cell_specimen_id'
merged_df = coords_df.merge(patchseq_df[['cell_specimen_id', 'cellType']], on='cell_specimen_id', how='left')
merged_df['cellType'].fillna(0, inplace=True)

# Assign colors based on cellType
color_map = {1: (0, 0, 1, 1), 2: (1, 0, 0, 1), 0: (0.5, 0.5, 0.5, 1)}
merged_df['color'] = merged_df['cellType'].map(color_map)

# Convert to numpy arrays
voxel_coordinates = merged_df[['x', 'y', 'z']].values
color_values = np.array(merged_df['color'].tolist())

# Debugging output
print("Merged dataframe preview:")
print(merged_df[['x', 'y', 'z', 'cell_specimen_id', 'cellType', 'color']].head())
print(f"Number of points to plot: {voxel_coordinates.shape[0]}")
print(f"Unique cell types in data: {merged_df['cellType'].unique()}")

# Initialize Napari viewer
viewer = napari.Viewer(ndisplay=3)
viewer.add_points(
    voxel_coordinates,
    size=2,
    face_color=color_values,
    name="Voxel Coordinates"
)

# Load and add multiple meshes
for mesh_name, mesh_file in MESH_FILES.items():
    print(f"Loading mesh: {mesh_name} from {mesh_file}...")
    mesh = trimesh.load(mesh_file)
    if mesh.is_empty:
        raise ValueError(f"The file '{mesh_file}' contains no valid mesh data.")
    print(f"{mesh_name} loaded successfully.")

    mesh_vertices = np.array(mesh.vertices)
    if mesh_name == "MouseBrainAllen3 Mesh":
        print("Calibrating MouseBrainAllen3 mesh from µm to 25 µm voxel space...")
        mesh_vertices /= 25.0

#you can change color of the meshes in napari
    mesh_faces = np.array(mesh.faces)
    viewer.add_surface(
        (mesh_vertices, mesh_faces),
        name=mesh_name,
        colormap="GrBu",
        opacity=0.5
    )

# Start Napari
print("Starting Napari viewer...")
napari.run()


Loading coordinates from /Users/yoh/Downloads/250121_Brians_coordinates2.csv...
Loading patchSeqCells data from /Users/yoh/Downloads/patchSeqCells.xlsx...
Merged dataframe preview:
            x           y       z  cell_specimen_id  cellType         color
0  422.764570  165.806602  180.72        1212569320       1.0  (0, 0, 1, 1)
1  412.516328  169.914746  181.46        1216310187       1.0  (0, 0, 1, 1)
2  397.679336  173.808711  196.96        1221572804       2.0  (1, 0, 0, 1)
3  420.114180  174.024590  186.04        1246112075       2.0  (1, 0, 0, 1)
4  420.056328  170.107988  201.36        1274322564       2.0  (1, 0, 0, 1)
Number of points to plot: 113
Unique cell types in data: [1. 2. 0.]
Loading mesh: LC Mesh from /Users/yoh/Library/CloudStorage/OneDrive-AllenInstitute/data/LC_mask_2/maskv1/LC_ccf_v1_250102.obj...
LC Mesh loaded successfully.
Loading mesh: subCD Mesh from /Users/yoh/Library/CloudStorage/OneDrive-AllenInstitute/data/LC_mask_2/maskv1/subCD_ccf_v1_250102.obj...
su

In [6]:
import os
import pandas as pd
import numpy as np
import napari
import trimesh
import matplotlib.pyplot as plt

# Define file paths
BASE_DIR = os.path.abspath("/Users/yoh/Library/CloudStorage/OneDrive-AllenInstitute/data")
COORDINATES_FILE = os.path.abspath("/Users/yoh/Downloads/250121_Brians_coordinates2.csv")
EPHYS_FILE = os.path.abspath("/Users/yoh/Downloads/IVSCC_LC_DR_ephys_snakemake_240209.csv")

MESH_FILES = {
    "LC Mesh": os.path.join(BASE_DIR, "LC_mask_2/maskv1/LC_ccf_v1_250102.obj"),
    "subCD Mesh": os.path.join(BASE_DIR, "LC_mask_2/maskv1/subCD_ccf_v1_250102.obj"),
    "subCV Mesh": os.path.join(BASE_DIR, "LC_mask_2/maskv1/subCV_ccf_v1_250102.obj"),
    "MouseBrainAllen3 Mesh": os.path.abspath("/Users/yoh/Desktop/Allen_CCF/MouseBrainAllen3.obj"),
}

# Load the coordinate data
print(f"Loading coordinates from {COORDINATES_FILE}...")
coords_df = pd.read_csv(COORDINATES_FILE)
coords_df = coords_df.dropna(subset=['x', 'y', 'z'])  # Remove rows with missing values
coords_df.columns = coords_df.columns.str.strip()  # Remove trailing spaces
coords_df[['x', 'y', 'z']] /= 25.0  # Convert to voxel space

# Load the ephys data
print(f"Loading ephys data from {EPHYS_FILE}...")
ephys_df = pd.read_csv(EPHYS_FILE)

# Merge coordinates with ephys data
merged_df = coords_df.merge(ephys_df[['cell_specimen_id', 'width_short_square']], on='cell_specimen_id', how='inner')
merged_df.dropna(subset=['width_short_square'], inplace=True)

# Check if the merged dataframe is empty
if merged_df.empty:
    raise ValueError("No matching data found between coordinates and ephys files.")

# Normalize the 'width_short_square' values
min_width = merged_df['width_short_square'].min()
max_width = merged_df['width_short_square'].max()
merged_df['normalized_color'] = (merged_df['width_short_square'] - min_width) / (max_width - min_width)

# Convert to numpy arrays
voxel_coordinates = merged_df[['x', 'y', 'z']].values
colormap = plt.get_cmap("Reds")
color_values = colormap(merged_df['normalized_color'].values)[:, :3]  # Get RGB values

# Debugging output
print("Merged dataframe preview:")
print(merged_df[['x', 'y', 'z', 'width_short_square', 'normalized_color']].head())
print(f"Number of points to plot: {voxel_coordinates.shape[0]}")
print(f"Color values range: {merged_df['normalized_color'].min()} to {merged_df['normalized_color'].max()}")

# Initialize Napari viewer
viewer = napari.Viewer(ndisplay=3)
viewer.add_points(
    voxel_coordinates,
    size=2,
    face_color=color_values,
    name="Voxel Coordinates"
)

# Load and add multiple meshes
for mesh_name, mesh_file in MESH_FILES.items():
    print(f"Loading mesh: {mesh_name} from {mesh_file}...")
    mesh = trimesh.load(mesh_file)
    if mesh.is_empty:
        raise ValueError(f"The file '{mesh_file}' contains no valid mesh data.")
    print(f"{mesh_name} loaded successfully.")

    mesh_vertices = np.array(mesh.vertices)
    if mesh_name == "MouseBrainAllen3 Mesh":
        print("Calibrating MouseBrainAllen3 mesh from µm to 25 µm voxel space...")
        mesh_vertices /= 25.0

    mesh_faces = np.array(mesh.faces)
    viewer.add_surface(
        (mesh_vertices, mesh_faces),
        name=mesh_name,
        colormap="GrBu",
        opacity=0.5
    )

    
    
# Start Napari
print("Starting Napari viewer...")
napari.run()


Loading coordinates from /Users/yoh/Downloads/250121_Brians_coordinates2.csv...
Loading ephys data from /Users/yoh/Downloads/IVSCC_LC_DR_ephys_snakemake_240209.csv...
Merged dataframe preview:
            x           y           z  width_short_square  normalized_color
1  412.516328  169.914746  181.460000             0.00142          0.407186
2  397.679336  173.808711  196.960000             0.00056          0.149701
3  420.114180  174.024590  186.040000             0.00068          0.185629
4  420.056328  170.107988  201.360000             0.00046          0.119760
5  403.197891  171.716738  184.589023             0.00048          0.125749
Number of points to plot: 63
Color values range: 0.0 to 1.0
Loading mesh: LC Mesh from /Users/yoh/Library/CloudStorage/OneDrive-AllenInstitute/data/LC_mask_2/maskv1/LC_ccf_v1_250102.obj...
LC Mesh loaded successfully.
Loading mesh: subCD Mesh from /Users/yoh/Library/CloudStorage/OneDrive-AllenInstitute/data/LC_mask_2/maskv1/subCD_ccf_v1_250102.obj..